In [ ]:
!python3 -V

Python 3.8.15


In [ ]:
!pip3 install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers
!pip install fastai==2.2.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1 MB 1.4 MB/s eta 0:03:59tcmalloc: large alloc 1147494400 bytes == 0x3a7b6000 @  0x7f44e7ef3615 0x5d631c 0x51e4f1 0x51e67b 0x4f7585 0x49ca7c 0x4fdff5 0x49caa1 0x4fdff5 0x49ced5 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x5d7c18 0x5d9412 0x586636 0x5d813c 0x55f3fd 0x55e571 0x5d7cf1 0x49ced5 0x55e571 0x5d7cf1 0x49ec69 0x5d7c18 0x49ca7c 0x4fdff5 0x49ced5
     |█████████████████████████████▏  | 1055.7 MB 1.2 MB/s eta 0:01:28tcmalloc: large alloc 1434370048 bytes == 0x7ee0c000 @  0x7f44e7ef3615 0x5d631c 0x51e4f1 0x51e67b 0x4f7585 0x49ca7c 0x4fdff5 0x49caa1 0x4fdff5 0x49ced5 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x5d7c18 0x5d9412 0x586636 0x5d813c 0x55f3fd 0x55e571 0x5d7cf1 0x49ced5 0x55e571 0x5d7cf1 0x49ec69 0x5d7c18 0x49ca7c 0x4fdff5 0x49ced5
     |█████

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from fastai.text.all import *
import gc

In [ ]:
print(torch. __version__)

1.7.1+cu110


In [ ]:
pretrained_weights = 'gpt2'
tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_weights)
model = GPT2LMHeadModel.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value) # Python
    if use_cuda: 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False
#Remember to use num_workers=0 when creating the DataBunch.

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/YelpDataset/dataframes

AutoDF.pkl		     HealthMedicalDF.pkl  restaurantDF.pkl
beautyandSpaDF.pkl	     HomeServicesDF.pkl   shoppingDF.pkl
EventPlanningServicesDF.pkl  LocalServicesDF.pkl
foodDF.pkl		     NightlifeDF.pkl


In [ ]:
%cp /content/drive/MyDrive/YelpDataset/dataframes/shoppingDF.pkl /content/

In [ ]:
import pandas as pd
df = pd.read_pickle('shoppingDF.pkl')

In [ ]:
df = df.rename(columns={'text': 'reviewText'})
df = df.rename(columns={'stars': 'overall'})
df = df.rename(columns={'categories': 'category'})

In [ ]:
df = df[['business_id','name','city','state','overall','category','reviewText']]

In [ ]:
df.head(1)

,business_id,name,city,state,overall,category,reviewText
91810,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,Brentwood,MO,2.5,Shopping,This has become my go to place for shoes. I am a rewards member and they often send me coupons and notices about their sales. The best sale they have is the BOGO 1/2 off plus 15% off. That is a great time to stock up on shoes especially if you have a coupon on top of that. \n\nOther reviewers have complained about the return policy and I must say that I have never attempted to return anything at this store so I can't really comment on that. I guess that is just something to be aware of before making a purchase. \n\nThis store also may not have your size for the shoe you want but the...


In [ ]:
df['sentenceLength'] = df['reviewText'].str.count(' ') + 1

In [ ]:
df.shape

(338700, 8)

In [ ]:
df.head(2)

,business_id,name,city,state,overall,category,reviewText,sentenceLength
91810,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,Brentwood,MO,2.5,Shopping,This has become my go to place for shoes. I am a rewards member and they often send me coupons and notices about their sales. The best sale they have is the BOGO 1/2 off plus 15% off. That is a great time to stock up on shoes especially if you have a coupon on top of that. \n\nOther reviewers have complained about the return policy and I must say that I have never attempted to return anything at this store so I can't really comment on that. I guess that is just something to be aware of before making a purchase. \n\nThis store also may not have your size for the shoe you want but the...,196
91811,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,Brentwood,MO,2.5,Shopping,"Oh, I do enjoy Famous Footwear. \n\nThe occasional bogo 50% off.\n\nThe clearance is impressive; I usually get a pretty good deal (though they recently redid this area and it's more difficult to shop now because it's hard to see the shoes). \n\nThe associates at this location are ideal. They don't interrupt your shopping experience to ask if you need anything when you clearly are doing just fine. They get you through the register in a jiffy.\n\nDefinitely join the rewards program - I have a $5 credit every 2nd or 3rd time I go there. I believe they send a birthday reward, too.",102


In [ ]:
sample_df = df.sample(frac=0.05, random_state=2021).dropna().reset_index()

In [ ]:
sample_df.shape

(16935, 9)

In [ ]:
del df
gc.collect()

37

In [ ]:
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]

In [ ]:
learn = Learner(dls=None, model=model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity()).to_fp16()

### Generated examples of Toys_and_Games_5

In [ ]:
!ls /content/drive/MyDrive/YelpDataset/models

gpt2-finetuned-yelp-reviews-sample-1cycle-Automotive.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-BeautySpa.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-Food.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-HomeServices.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-Restaurants.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-Shopping.pth


In [ ]:
%cp /content/drive/MyDrive/YelpDataset/models/gpt2-finetuned-yelp-reviews-sample-1cycle-Shopping.pth /content/

In [ ]:
learn.load("gpt2-finetuned-yelp-reviews-sample-1cycle-Shopping").cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [ ]:
"""N_INITIAL_WORDS = 5
LEN_SAMPLE_DF = sample_df.shape[0]
N_REVIEWS = 10
BINS = list(range(10,15))"""
N_INITIAL_WORDS = 5
LEN_SAMPLE_DF = sample_df.shape[0]
N_REVIEWS = 2000
BINS = list(range(10,351))

In [ ]:
import warnings
warnings.filterwarnings(action="ignore")
from sklearn.model_selection import train_test_split
generated_df_rows = []
for i in range(len(BINS)-1):
    sample_row_df = sample_df[(sample_df["sentenceLength"]>=BINS[i]) & (sample_df["sentenceLength"]<BINS[i+1])]
    bin_proportion = (sample_row_df.shape[0])/float(LEN_SAMPLE_DF)
    n_reviews_to_generate = int((bin_proportion)*N_REVIEWS)+1
    try:
        or_df, cg_df =  train_test_split(sample_row_df, test_size=0.5, random_state=2021)
    except Exception as e:
        print(e)
        continue
    sample_reviews_df = cg_df.sample(n=n_reviews_to_generate,random_state=2021)
    or_sample_reviews_df = or_df.sample(n=n_reviews_to_generate,random_state=2021)
    if BINS[i] in range(10,21):
        max_len = 20
    else:
        max_len = BINS[i]
    for index, row in sample_reviews_df.iterrows():
        text = row["reviewText"]
        prompt = (" ").join(text.strip().split(" ")[:N_INITIAL_WORDS])
        prompt_ids = tokenizer.encode(prompt)
        inp = tensor(prompt_ids)[None].cuda()
        # set seed to reproduce results
        random_seed(seed_value=2021, use_cuda=True)
        preds = learn.model.generate(inp, max_length=max_len, do_sample=True, top_k=0, top_p=0.92, temperature=0.7)
        decoded_preds = tokenizer.decode(preds[0],skip_special_tokens=True)
        generated_df_rows.append([row["category"],row["overall"],decoded_preds,"CG"])
    for index, row in or_sample_reviews_df.iterrows():
        generated_df_rows.append([row["category"],row["overall"],row["reviewText"],"OR"])
#     break
generated_df = pd.DataFrame(generated_df_rows, columns=["category", "rating", "text", "label"])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

In [ ]:
generated_df.shape

(4196, 4)

In [ ]:
generated_df.head()

,category,rating,text,label
0,Shopping,4.5,I would not recommend this location to anyone. \n\nI have visited a few other locations.,CG
1,Shopping,5.0,Many many thanks for making this an easy and fun experience for all. Thank you!Very rude.,CG
2,Shopping,4.5,"Great selection and very knowledgeable staff, best place for cigars!",OR
3,Shopping,3.0,great title with average touristy trap for mardi gras goers,OR
4,Shopping,4.5,Good local shop. Its kind of a shame they don't carry it. If you have a question,CG


In [ ]:
def post_process(str, k=4):
    if not str.endswith("."):
        full_stop_split_str = str.split(".")
        clean_full_stop_split_str = [s.strip() for s in full_stop_split_str if s.strip()]
        if len(clean_full_stop_split_str) > 1:
            last_sentence = clean_full_stop_split_str[-1]
        else:
            return str
        last_sentence_len = len(last_sentence.split(" "))
        if last_sentence_len > k:
            return str
        return (". ").join(clean_full_stop_split_str[:-1]).strip()+(".")
    return str

In [ ]:
generated_df['text_'] = generated_df['text'].apply(lambda x: post_process(x))

In [ ]:
generated_df.head()

,category,rating,text,label,text_
0,Shopping,4.5,I would not recommend this location to anyone. \n\nI have visited a few other locations.,CG,I would not recommend this location to anyone. \n\nI have visited a few other locations.
1,Shopping,5.0,Many many thanks for making this an easy and fun experience for all. Thank you!Very rude.,CG,Many many thanks for making this an easy and fun experience for all. Thank you!Very rude.
2,Shopping,4.5,"Great selection and very knowledgeable staff, best place for cigars!",OR,"Great selection and very knowledgeable staff, best place for cigars!"
3,Shopping,3.0,great title with average touristy trap for mardi gras goers,OR,great title with average touristy trap for mardi gras goers
4,Shopping,4.5,Good local shop. Its kind of a shame they don't carry it. If you have a question,CG,Good local shop. Its kind of a shame they don't carry it. If you have a question


In [ ]:
generated_df.drop(columns=["text"],axis=1,inplace=True)

In [ ]:
generated_df[1500:1512]

,category,rating,label,text_
1500,Shopping,4.5,CG,"Me and my fiance visited this store and the girl in the front was very rude and rude. When she asked us if we wanted a ""truck"" we told her we were going to go to another store and we would be able to use them. She seemed to be annoyed that we were going to use them"
1501,Shopping,4.0,CG,"Always had great experiences here, especially with the guys at the front desk. They were very friendly and helpful. If you are looking for a great deal, this is the place to go. I found the deals to be a bit steep for what I paid for. The staff was very helpful."
1502,Shopping,4.5,CG,The staff was amazing and friendly. The only reason I didn't give them a star is because they were so busy. The store itself is small but the atmosphere is so great. The store is very well organized and clean. I would recommend coming here if you are looking for a new mattress.
1503,Shopping,4.5,CG,I am a very fussy woman and I am very comfortable with my body shape. I have always been a very nervous woman and I am not that nervous at all. I have been going to this place for years and I have been a loyal customer for years. I have always had great customer service.
1504,Shopping,4.5,CG,What a wonderful service! I had an appointment with them and was able to make a reservation through the app. The woman was very patient and took the time to explain to me how to use the app and what I needed to do to get there. I highly recommend this place!I love this store.
1505,Shopping,3.0,CG,"This is the most welcoming and supportive store I have ever been in. The staff is friendly and helpful. If you are looking for a unique and unique gift or decor, you should go here. They have everything from vintage to modern, from greeting cards, candles, etc. The staff is always willing to answer any"
1506,Shopping,4.5,CG,The selection is beautiful but not overwhelming at all. I saw a lot of the stuff that I wanted and felt very comfortable. There were some things that I had a hard time finding. I found some really good items that were a little over priced. There was also a little bit of a line
1507,Shopping,3.5,CG,"I Love Ikea!! Every time I go in there they are always friendly and helpful! If you have an issue with Ikea, they will get the job done quickly and get it right. I have been in many Ikea stores and have never had an issue. The customer service is always"
1508,Shopping,1.5,CG,"Four hours late. Nothing but a bunch of excuses and excuses. I called the store and spoke to an associate. She said, ""well, the manager has to call you back"" and then proceeded to call me back. I then called the store and spoke to the manager."
1509,Shopping,3.5,CG,"My experience at LensCrafters was a little different. \n\nI first visited LensCrafters while in a pair of glasses repair stores, and since then, I have had a lot of eye wear and contact lens issues. I am extremely pleased with the service provided by the LensCrafters staff."


In [ ]:
generated_df.to_csv("Generated_Sentences_Shopping.csv", index=None)

In [ ]:
%cp /content/Generated_Sentences_Shopping.csv /content/drive/MyDrive/YelpDataset/generatedData